In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense
import mlflow
import mlflow.tensorflow

In [ ]:
from tensorflow.keras.models import load_model
import joblib

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

#autolog
mlflow.tensorflow.autolog()

In [ ]:
data = yf.download("AAPL", start="2015-01-01", end="2024-12-01")

In [ ]:
close_prices = data['Close'].values

In [ ]:
# Normalize
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(close_prices)

In [ ]:
# Prepare sequences
sequence_length = 60
X, y = [], []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i, 0]) # "0" 1D
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
from tensorflow.keras.optimizers import Adam
with mlflow.start_run(run_name="LSTM_Stock_PredictorFinal"):
    epochs = 20
    batch_size = 32
    # Logging before training
    mlflow.log_param("sequence_length", sequence_length)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)

        # Build LSTM
    model = Sequential([
        Input(shape=(X.shape[1], 1)),     
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train model (autolog will track metrics)
    Model = model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)


    preds = model.predict(X)
    preds = scaler.inverse_transform(preds)

    comparison_df = data.iloc[sequence_length:].copy()
    comparison_df['Predicted_Close'] = preds

    result = comparison_df[['Close', 'Predicted_Close']]
    result.to_csv("predictions.csv", index=False)
    
    #Lgging after traning
    for epoch, loss in enumerate(Model.history['loss']):
        mlflow.log_metric("train_loss", loss, step=epoch)
        
    mlflow.log_artifact("Path_To_Artifact_predictions.csv")